In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
disk_usage = dataiku.Dataset("disk_usage")
df = disk_usage.get_dataframe()

In [0]:
# Write recipe outputs
json = dataiku.Dataset("Json")
json.write_with_schema(df)